# Laboratorium 1 - content-based recommender

## Przygotowanie

 * dataset i potrzebne biblioteki są dokładnie takie same jak na poprzednim laboratorium
 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from sklearn.model_selection import train_test_split, KFold

In [2]:
# liczba parametrow opisujacych filmy i uzytkownikow zalezy tylko od nas
K = 20
#TUTAJ MODYFIKOWAĆ W CELY SPRAWDZANIA WYNIKÓW

In [3]:
# wczytujemy oceny uytkownikow i od razu dzielimy je na dwa zbiory - treningowy i testowy

all_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
train_ratings_set, test_ratings_set = train_test_split(all_ratings, test_size=0.05)
train_ratings_set

,userId,movieId,rating
88432,571,1326,2.0
26271,182,1215,4.5
75688,477,2746,3.5
59642,387,2664,4.0
95746,600,7084,3.5
...,...,...,...
1571,16,111,4.5
88909,573,52712,0.5
82210,522,62,3.0
65209,418,367,0.5


In [4]:
# inicjalizujemy macierz preferencji uzytkownikow liczbami losowymi z przedzialu [0.0, 5.0]

def initialize_users(raw_ratings, k):
    users_no = raw_ratings['userId'].unique().size
    users = pandas.DataFrame(5.0 * np.random.uniform(size=(users_no, k)), index=raw_ratings['userId'].unique(), columns=['x%s' % i for i in range(k)])
    users.sort_index(inplace=True) 
    return users_no, users

users_no, users = initialize_users(train_ratings_set, K)
users

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19
1,2.577299,0.015318,1.426966,4.377501,1.792831,1.910550,1.040855,3.218975,0.154309,2.769925,0.401223,3.791374,2.855636,2.568787,3.843912,4.112973,3.202030,0.482851,2.703050,1.686654
2,4.163496,0.625204,3.855524,2.850812,2.170905,2.549453,0.972591,1.200414,3.908373,4.632526,1.088416,2.331222,4.541389,0.955519,4.985853,3.159198,4.399677,1.053524,0.403494,3.871044
3,1.798516,1.406947,1.069378,0.106150,0.480183,3.907184,1.518664,2.924685,4.596193,1.277703,3.826826,0.890826,4.994693,3.555310,0.617393,0.627148,2.531505,3.183526,4.937652,1.630100
4,4.382491,0.173269,2.328077,4.836375,0.179145,3.278606,1.829154,4.383413,2.038579,3.622202,2.909046,0.758704,2.150523,4.023521,2.328461,3.921801,2.244328,0.765161,3.851934,3.378422
5,1.755486,0.696513,1.950219,2.426013,3.558994,1.520323,2.859381,0.537024,3.405013,0.609235,0.151328,4.608550,3.355527,4.101497,0.784414,0.230364,2.709478,2.102270,3.663422,3.663077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,1.937536,2.873522,0.058606,1.243448,3.334018,2.529984,1.245749,2.116161,2.860751,2.084129,4.223993,1.778332,4.099628,2.488175,1.611383,4.189690,0.036061,2.695880,0.671745,2.554014
607,4.527081,4.221603,3.550263,3.811737,4.736206,1.621456,2.644777,2.489447,0.090839,3.487393,3.346978,0.666253,3.132880,4.599222,3.555907,4.936049,4.097597,0.742586,4.773283,4.784826
608,0.036925,4.504110,3.077474,4.630641,4.988153,3.699607,2.254576,2.043717,1.560464,1.117410,2.088083,0.973661,3.866896,1.447375,4.391465,4.881463,3.071129,2.883725,4.384680,3.073436
609,4.531983,0.933272,3.534276,1.111300,0.565528,0.222158,2.589726,2.451981,0.484953,0.120989,0.086502,0.334832,0.891123,1.199769,1.447444,4.061416,2.896162,2.159017,0.728803,3.937497


In [5]:
# inicjalizujemy macierz cech filmow liczbami losowymi z przedzialu [0.0, 1.0]

def initialize_movies(raw_ratings, k):
    movies_no = raw_ratings['movieId'].unique().size
    movies = pandas.DataFrame(np.random.uniform(size=(movies_no, k)), index=raw_ratings['movieId'].unique(), columns=['x%s' % i for i in range(k)])
    movies.sort_index(inplace=True) 
    return movies_no, movies

movies_no, movies = initialize_movies(train_ratings_set, K)
movies

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19
1,0.595335,0.132188,0.215080,0.600698,0.150714,0.988410,0.405253,0.670715,0.632004,0.080519,0.473050,0.707095,0.636835,0.169563,0.831899,0.385028,0.292616,0.940312,0.735877,0.018216
2,0.265182,0.890429,0.336918,0.031269,0.845232,0.763410,0.763381,0.736427,0.790908,0.020707,0.166593,0.317127,0.930095,0.599557,0.848550,0.330738,0.225706,0.917768,0.832743,0.674044
3,0.876799,0.890843,0.776494,0.983069,0.850782,0.140665,0.534919,0.289247,0.040633,0.836114,0.338368,0.364214,0.685031,0.296066,0.041091,0.912607,0.397296,0.086694,0.422360,0.751221
4,0.348015,0.595281,0.235108,0.157953,0.006191,0.709447,0.554189,0.893051,0.964830,0.277229,0.091477,0.083466,0.575118,0.810110,0.449573,0.546081,0.720331,0.735733,0.032807,0.832723
5,0.030248,0.077207,0.214888,0.444164,0.306353,0.744802,0.657265,0.103172,0.907004,0.467930,0.800771,0.395433,0.541313,0.631163,0.726185,0.967686,0.663882,0.736837,0.537357,0.169375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.096613,0.345061,0.373278,0.570162,0.987132,0.982455,0.752373,0.316138,0.090262,0.056618,0.468389,0.390804,0.283715,0.653932,0.546936,0.284216,0.535053,0.565334,0.632910,0.708459
193583,0.579777,0.635219,0.287644,0.126776,0.152760,0.978915,0.716384,0.611713,0.577622,0.681763,0.236787,0.835334,0.625195,0.542307,0.364198,0.265613,0.437980,0.308352,0.685739,0.376861
193585,0.676887,0.837011,0.405821,0.284817,0.611962,0.022393,0.189851,0.090845,0.545930,0.236804,0.021281,0.207158,0.167521,0.583364,0.357375,0.033991,0.630383,0.394674,0.410080,0.534913
193587,0.420013,0.097460,0.232639,0.307014,0.315230,0.069083,0.099417,0.309266,0.596410,0.968456,0.643068,0.428113,0.824645,0.028509,0.893542,0.047479,0.484933,0.998099,0.751304,0.246822


In [79]:
# za pomoca sprytnej sztuczki przeksztalcamy oceny z formatu dostarczonego przez MovieLens do uzytecznej macierzy
# zwroc uwage na to, ze czesci filmow i uzytkownikow moze brakowac po podziale datasetu na dwie czesci
#   - byc moze warto uzupelnic brakujace kolumny i wiersze

def get_ratings(raw_ratings, movies, nan=False):
    ratings = raw_ratings.pivot(*raw_ratings.columns)
    if not nan:
        ratings = ratings.fillna(0.0)
    # ...
    missing_movies = set(movies.index).difference(set(raw_ratings['movieId']))
    for movie in missing_movies:
        ratings[movie] = 0.0
    ratings = ratings.reindex(sorted(ratings.columns), axis=1)
    return ratings

ratings = get_ratings(train_ratings_set, movies)
ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Część 2. - trening modelu

In [80]:
# trenujemy model iteracyjnie, wykorzystujac gradient descent

alpha = 0.00003 # learning speed
delta = 1000 # minimal upgrade for each step
lambd = 0.01 # regularization weight

def calculate_user_preferences(users, movies, ratings, raw_ratings, users_no, movies_no, alpha, delta, lambd):
    total_error = 0.0
    users_model = users.copy()
    movies_model = movies.copy()
    
    while(True):
        previous_total_error = total_error

        predicted_ratings = np.dot(users_model, movies.T)# ...
        errors = np.where(ratings==0.0, pandas.DataFrame(np.zeros((users_no, movies_no))), predicted_ratings - ratings)
        users_gradient = np.dot(errors, movies)# ...
        movies_gradient = np.dot(errors.T, users)# ...
#         print(users_gradient.shape)
#         print(movies_gradient.shape)
        
        # zauwaz, ze nie uzywamy biasow i nie potrzebujemy dodatkowej macierzy do regularyzacji
        #  - wystarczy, ze uzyjemy odpowiednio macierzy users_model i movies_model
        
        # musimy zaktualizowac dwa modele
        
        users_model = users_model - alpha * (users_gradient + lambd * users_model)# ...
        movies_model = movies_model - alpha * (movies_model + lambd * movies_model)# ...

        total_error = np.sum(errors ** 2)
        print(total_error)
        progress = abs(previous_total_error - total_error)
        if progress < delta:
            break
            
    return users_model, movies_model

users_model, movies_model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)

48609830.69269533
40848738.49517248
35472991.24851375
31487600.76556252
28382793.561327547
25876569.308432937
23800533.169250015
22046943.889519174
20542792.83860744
19236315.793353766
18089508.608456135
17073707.20916569
16166826.647945425
15351552.638953827
14614110.452400725
13943400.773436172
13330378.196611945
12767595.366794046
12248863.145620795
11768993.745341476
11323604.205551144
10908964.387885464
10521878.220978051
10159590.053206688
9819710.154516477
9500154.958371839
9199098.74916308
8914934.310606899
8646240.645675438
8391756.319466243
8150357.3058251925
7921038.466565156
7702897.980281561
7495124.18152848
7296984.381729489
7107815.3288794
6927015.029905087
6754035.71199238
6588377.740600647
6429584.344801276
6277237.026888807
6130951.554373085
5990374.449571122
5855179.905929276
5725067.071575731
5599757.649942468
5478993.775002737
5362536.12506058
5250162.244348896
5141665.046141446
5036851.474818768
4935541.307473129
4837566.078298474
4742768.111267556
4650999.6485185

## Część 3. - podobieństwo elementów

In [81]:
# przygotujmy funkcje obliczajaca odleglosc cosinusowa miedzy kazda para elementow (filmow lub uzytkownikow)
def cosine_similarity(vectors):
    # przydadza nam sie dlugosci wektorow
#     vectors = np.ma.array(vectors, mask=np.isnan(vectors)) #TO JEST CHYBA POTRZEBNE TYLKO W NASTĘPNYM KROKU
    lengths = np.linalg.norm(vectors, axis=1)
    # podobienstwo liczymy w dwoch krokach - najpierw liczymy iloczyn skalarny kazdej pary wektorow
    dot_products = vectors.dot(vectors.T)# ...
    # nastepnie dzielimy zarowno wiersze jak i kolumny przez dlugosci wektorow - przyda sie zmienna lengths oraz funkcja divide()
    similarity = dot_products.divide(lengths) # ...
    similarity = similarity.divide(lengths.T)
    return similarity

cosine_similarity(movies_model)

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,1.000000,0.719820,0.531219,0.734381,0.820027,0.676321,0.767685,0.732405,0.743529,0.818378,...,0.712430,0.640902,0.809525,0.836955,0.698328,0.793696,0.860895,0.785804,0.840538,0.740041
2,0.941237,1.000000,0.689472,0.957206,0.867864,0.792948,0.953636,0.836952,0.919723,0.855581,...,0.864101,0.795972,0.901511,1.055996,0.750381,1.003749,0.988870,1.167226,0.869465,0.833050
3,0.630908,0.626230,1.000000,0.669713,0.669152,0.827102,0.907912,0.686800,0.623922,0.894061,...,0.953796,0.796103,0.847623,1.008256,0.651546,0.802419,0.801635,1.095420,0.693080,0.777881
4,0.748368,0.745974,0.574632,1.000000,0.773158,0.649395,0.760728,0.695558,0.706575,0.841238,...,0.699830,0.697062,0.672604,0.929065,0.695105,0.753313,0.848397,0.977809,0.669871,0.726268
5,0.862705,0.698249,0.592743,0.798195,1.000000,0.825486,0.876123,0.802847,0.734875,0.801918,...,0.747075,0.662760,0.727240,0.853767,0.733631,0.841812,0.828485,0.835632,0.830177,0.799023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.752136,0.727432,0.640252,0.700526,0.758269,0.752900,0.842355,0.769994,0.729908,0.852630,...,0.833240,0.684829,0.740001,0.799414,0.610525,1.000000,0.784546,0.926557,0.639878,0.765035
193583,0.825196,0.724887,0.646980,0.798017,0.754844,0.738562,0.795149,0.809805,0.751138,0.824669,...,0.818509,0.698053,0.734978,0.919393,0.642195,0.793566,1.000000,0.933882,0.744809,0.770547
193585,0.464113,0.527216,0.544750,0.566721,0.469127,0.541369,0.570389,0.489614,0.452889,0.646286,...,0.535693,0.532882,0.537427,0.746356,0.410457,0.577482,0.575432,1.000000,0.530418,0.563861
193587,0.769119,0.608433,0.533982,0.601497,0.722058,0.573968,0.685184,0.597779,0.552359,0.688196,...,0.521445,0.582770,0.684507,0.930171,0.598917,0.617861,0.711007,0.821760,1.000000,0.815757


In [110]:
nan_ratings = get_ratings(train_ratings_set, movies, True)

In [104]:
def my_norm(vectors):
    return np.sqrt(np.sum(vectors ** 2, axis=1))


def my_cosine_similarity(vectors):
    # przydadza nam sie dlugosci wektorow
#     vectors = np.ma.array(vectors, mask=np.isnan(vectors)) #TO JEST CHYBA POTRZEBNE TYLKO W NASTĘPNYM KROKU
    lengths = my_norm(vectors)
    # podobienstwo liczymy w dwoch krokach - najpierw liczymy iloczyn skalarny kazdej pary wektorow
    dot_products = vectors.dot(vectors.T)# ...
    # nastepnie dzielimy zarowno wiersze jak i kolumny przez dlugosci wektorow - przyda sie zmienna lengths oraz funkcja divide()
    
    similarity = np.divide(dot_products, lengths) # ...
    similarity = np.divide(similarity, lengths.T)
    return similarity


print(my_cosine_similarity(nan_ratings.T))

[[0.9999999999999999 0.6275939427930455 0.676355748373102 ... -- -- --]
 [0.2562204243626927 1.0 0.39132321041214757 ... -- -- --]
 [0.11898946725690734 0.16862965040194428 1.0 ... -- -- --]
 ...
 [-- -- -- ... 1.0 1.0 --]
 [-- -- -- ... 1.0 1.0 --]
 [-- -- -- ... -- -- 1.0]]


In [42]:
# teraz mozemy znalexc k elementow najbardziej podobnych do danego

def k_most_similar(vectors, i, k):
    sim_matrix = cosine_similarity(vectors)
    ith = sim_matrix[i]
    # przyda sie funkcja np.argsort()
#     ar = np.argsort(vectors)
#     return vectors[ar]# ...
    return vectors.argsort(ith)[:k]
#     return np.argsort(vectors)[ith].iloc[:k]# ...
# (ith)

# k_most_similar(movies_model, 193587, 8)

## Część 4. - Item2Item collaborative filtering

In [112]:
# sprobujmy innego podejscia - Item2Item CF przewiduje rating tylko na podstawie macierzy ratingow, bez koniecznosci trenowania
#   dodatkowych macierzy

# zauwaz, ze nie chcemy przeprowadzac obliczen tam, gdzie brakuje nam elementow
#   - oblicz macierz ratings z parametrem nan=True oraz wykorzystaj tzw. masked arrays: np.ma.array(x, mask=np.isnan(x))
#   w ten sposob unikniesz przeprowadzania niepotrzebnych obliczen
def predict_rating(row_sim, row_rate):
    sim = row_sim.copy()
    sim.mask = row_rate.mask
    return (sim.dot(row_rate.T)).sum() / sim.sum()


def item_to_item(ratings):
    masked_ratings = np.ma.array(ratings, mask=np.isnan(ratings))
    similarity = my_cosine_similarity(masked_ratings.T) # prawdopodobnie bedziesz musial zmodyfikowac te funkcje, by obslugiwala NaN
    print(similarity)
    sums = similarity.sum(axis=1)
    print(sums)
    model = np.array([[predict_rating(similarity_row, ratings_row) for similarity_row in similarity] for ratings_row in masked_ratings])
#     model = ratings.dot(similarity).divide(sums)# srednia ocen wystawionych przez uzytkownika wazona podobienstwem elementow
    print(model)
    return model


# print(item_to_item(nan_ratings))
i2i_model = item_to_item(nan_ratings)

[[0.9999999999999999 0.6275939427930455 0.676355748373102 ... -- -- --]
 [0.2562204243626927 1.0 0.39132321041214757 ... -- -- --]
 [0.11898946725690734 0.16862965040194428 1.0 ... -- -- --]
 ...
 [-- -- -- ... 1.0 1.0 --]
 [-- -- -- ... 1.0 1.0 --]
 [-- -- -- ... -- -- 1.0]]
[8725.111972297329 5340.243430141838 2792.7446360965987 ...
 61.29516792644877 61.29516792644877 4.739587526620651]


C:\Users\macie\AppData\Local\Temp/ipykernel_3156/57535020.py:10: RuntimeWarning: invalid value encountered in double_scalars
  return (sim.dot(row_rate.T)).sum() / sim.sum()


[[4.30879147 4.29792853 4.28957933 ... 5.         5.         4.23319326]
 [3.87763215 3.80252315 3.63969825 ... 4.25332703 4.25332703 3.79175263]
 [1.73353887 1.73600498 1.85189797 ...        nan        nan        nan]
 ...
 [2.96350145 2.95265459 2.8100075  ... 3.86951031 3.86951031 3.87015659]
 [3.21834101 3.22112669 3.15422722 ...        nan        nan 4.        ]
 [3.41156964 3.50951547 3.53619379 ... 3.99290313 3.99290313 3.95638512]]


## Część 5. - porównanie algorytmów

In [113]:
positive_threshold = 4.0
negative_threshold = 2.0

def calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold):
    # obliczamy true_positives itp.
    # nastepnie wszystkie metryki
    
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    
    for _, row in test_ratings_set.iterrows():
        
        my_prediction = predicted_ratings[row['movieId']][row['userId']]
        real_rating = row['rating']
 
        if real_rating >= positive_threshold and my_prediction >= positive_threshold:
            true_positives += 1
        elif real_rating >= positive_threshold and my_prediction <= positive_threshold:
            false_positives += 1
        elif real_rating <= negative_threshold and my_prediction <= negative_threshold:
            true_negatives += 1
        elif real_rating <= negative_threshold and my_prediction >= negative_threshold:
            false_negatives += 1
    
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = (2 * precision * recall) / (precision + recall)
        
    return {
        'true_positives': true_positives,
        'true_negatives': true_negatives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [114]:
# korzystając z funkcji z poprzedniego laboratorium, porownaj dwa zaimplementowane algorytmy Collaborative Filtering

print(i2i_model.shape)
print(movies.T.shape)

print(i2i_model)

predicted_ratings = np.dot(i2i_model, movies)
print(predicted_ratings)
calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold)

(610, 9554)
(20, 9554)
[[4.30879147 4.29792853 4.28957933 ... 5.         5.         4.23319326]
 [3.87763215 3.80252315 3.63969825 ... 4.25332703 4.25332703 3.79175263]
 [1.73353887 1.73600498 1.85189797 ...        nan        nan        nan]
 ...
 [2.96350145 2.95265459 2.8100075  ... 3.86951031 3.86951031 3.87015659]
 [3.21834101 3.22112669 3.15422722 ...        nan        nan 4.        ]
 [3.41156964 3.50951547 3.53619379 ... 3.99290313 3.99290313 3.95638512]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices